# Perceptrón: Implementación y Análisis
**Asignatura:** INFB6052 – Herramientas para la Ciencia de Datos\n
## Objetivo
Implementar un perceptrón simple para aprender funciones lógicamente linealmente separables (AND / OR), visualizar su proceso de entrenamiento y analizar resultados.
## Temas Clave
- Modelo de neurona artificial
- Función de activación escalón
- Regla de actualización de pesos\n
- Convergencia en problemas linealmente separables

## 1. Importar Librerías
Cargamos NumPy para operaciones numéricas, pandas para el dataset y matplotlib para visualizar métricas.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from perceptron import Perceptron
plt.style.use('seaborn-v0_8-whitegrid')

## 2. Cargar Dataset de Compuertas Lógicas
El dataset contiene ejemplos para AND y OR. Seleccionaremos una compuerta para entrenar.

In [ ]:
df = pd.read_csv('../data/logic_gates.csv')
df

## 3. Definir Funciones Auxiliares
Función para seleccionar compuerta y separar X (entradas) e y (etiquetas).

In [ ]:
def load_gate(gate: str):
    subset = df[df['gate']==gate].copy()
    X = subset[['x1','x2']].values
    y = subset['y'].values
    return X, y
X_and, y_and = load_gate('AND')
X_or, y_or = load_gate('OR')
X_and, y_and

## 4. Clase Perceptrón
La implementación está en el módulo `perceptron/perceptron.py`. A continuación mostramos la firma principal y descripción de sus métodos:
- `fit`: entrena y guarda historial de errores, accuracy y pesos.
- `predict`: devuelve clase binaria.
- `score`: accuracy sobre datos dados.

In [ ]:
from inspect import getsource
import perceptron.perceptron as pmod
print(getsource(pmod.Perceptron))

## 5. Entrenamiento
Entrenaremos el modelo sobre AND y OR para comparar convergencia.

In [ ]:
per_and = Perceptron(n_inputs=2, learning_rate=0.1, random_state=1)
hist_and = per_and.fit(X_and, y_and, epochs=15)
per_or = Perceptron(n_inputs=2, learning_rate=0.1, random_state=1)
hist_or = per_or.fit(X_or, y_or, epochs=15)
print('Accuracy AND:', per_and.score(X_and, y_and))
print('Accuracy OR:', per_or.score(X_or, y_or))
hist_and.errors, hist_or.errors

## 6. Gráficos de Métricas
Graficamos evolución de errores y accuracy por época.

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,4))
axes[0].plot(hist_and.errors, label='AND - Errores')
axes[0].plot(hist_or.errors, label='OR - Errores')
axes[0].set_xlabel('Época'); axes[0].set_ylabel('Errores'); axes[0].legend()
axes[1].plot(hist_and.accuracy, label='AND - Acc')
axes[1].plot(hist_or.accuracy, label='OR - Acc')
axes[1].set_xlabel('Época'); axes[1].set_ylabel('Accuracy'); axes[1].legend()
plt.tight_layout()
plt.show()

## 7. Frontera de Decisión
Función para visualizar la frontera aprendida en 2D.

In [ ]:
def plot_decision_boundary(model, X, y, ax, title):
    x_min, x_max = -0.5, 1.5
    y_min, y_max = -0.5, 1.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.3, cmap='coolwarm')
    ax.scatter(X[:,0], X[:,1], c=y, cmap='coolwarm', edgecolor='k', s=80)
    ax.set_title(title)
    ax.set_xlim(x_min, x_max); ax.set_ylim(y_min, y_max)
fig, axes = plt.subplots(1,2, figsize=(10,4))
plot_decision_boundary(per_and, X_and, y_and, axes[0], 'Frontera AND')
plot_decision_boundary(per_or, X_or, y_or, axes[1], 'Frontera OR')
plt.tight_layout(); plt.show()

## 8. Evolución de los Pesos
Mostramos cómo cambian los pesos (incluyendo bias) a lo largo de las épocas.

In [ ]:
def plot_weights(history, title):
    W = np.array(history.weights)
    plt.figure(figsize=(6,4))
    for i in range(W.shape[1]):
        plt.plot(W[:,i], label=f'w{i}')
    plt.xlabel('Época'); plt.ylabel('Valor'); plt.title(title); plt.legend(); plt.show()
plot_weights(hist_and, 'Pesos AND')
plot_weights(hist_or, 'Pesos OR')

## 9. Interpretación de Resultados
- El perceptrón converge rápidamente para AND y OR porque son linealmente separables.
- Los errores disminuyen hasta 0 y el accuracy alcanza 1.0 (100%).
- La frontera de decisión separa correctamente las clases.
- La evolución de pesos muestra estabilización una vez que no hay errores.
## 10. Trabajo Futuro
- Probar con problemas no linealmente separables (XOR) y observar fracaso.
- Extender a perceptrón multicapa (MLP) con backpropagation.